## Workflow
1. Configure `EDITO_API_TOKEN` and (optionally) `EDITO_DATA_BASE_URL`.
2. Tune the search filters below to target your area of interest.
3. Download one asset and visualize a slice of the dataset.

In [ ]:
import os
from pathlib import Path
from typing import Any, Dict, List, Optional

import matplotlib.pyplot as plt
import pandas as pd
import requests
import xarray as xr

%matplotlib inline
plt.style.use("seaborn-v0_8")

BASE_URL = os.getenv("EDITO_DATA_BASE_URL", "https://api.dive.edito.eu/data").rstrip("/")
API_TOKEN = os.getenv("EDITO_API_TOKEN")
DOWNLOAD_DIR = Path(os.getenv("EDITO_DOWNLOAD_DIR", "../data")).resolve()
DOWNLOAD_DIR.mkdir(parents=True, exist_ok=True)

session = requests.Session()
session.headers.update({"Accept": "application/json"})
if API_TOKEN:
    session.headers["Authorization"] = f"Bearer {API_TOKEN}"

BASE_URL, DOWNLOAD_DIR

In [ ]:
def fetch_json(path: str, params: Optional[Dict[str, Any]] = None, method: str = "GET", json_body: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Utility around requests to call EDITO endpoints."""
    url = path if path.startswith("http") else f"{BASE_URL}{path}"
    request_fn = session.get if method.upper() == "GET" else session.post
    response = request_fn(url, params=params, json=json_body, timeout=90)
    response.raise_for_status()
    return response.json()

def search_items(collections: Optional[List[str]] = None, bbox: Optional[List[float]] = None, datetime_range: Optional[str] = None, limit: int = 5) -> Dict[str, Any]:
    payload: Dict[str, Any] = {"limit": limit}
    if collections:
        payload["collections"] = collections
    if bbox:
        payload["bbox"] = bbox
    if datetime_range:
        payload["datetime"] = datetime_range
    return fetch_json("/search", method="POST", json_body=payload)

print("Session ready. Authenticated=", bool(API_TOKEN))

In [ ]:
collections_filter = ["climate_forecast-sea_water_potential_temperature"]
bbox_filter = [-10.0, 40.0, 10.0, 60.0]  # minLon, minLat, maxLon, maxLat
datetime_filter = "2024-10-01T00:00:00Z/2024-10-05T00:00:00Z"
search_response = search_items(collections=collections_filter, bbox=bbox_filter, datetime_range=datetime_filter, limit=5)
features = search_response.get("features", [])
summary_df = pd.DataFrame([
    {
        "id": feature.get("id"),
        "collection": feature.get("collection"),
        "datetime": feature.get("properties", {}).get("datetime"),
        "asset_keys": list(feature.get("assets", {}).keys()),
    }
    for feature in features
])
summary_df

In [ ]:
if not features:
    raise RuntimeError("No items found. Try relaxing the filters.")
feature = features[0]
assets = feature.get("assets", {})
asset_key = "data" if "data" in assets else next(iter(assets))
asset = assets[asset_key]
asset_url = asset.get("href")
asset_url

In [ ]:
def download_asset(url: str, chunk_size: int = 1024 * 1024) -> Path:
    """Stream an asset to disk to avoid loading everything in memory."""
    filename = url.split("/")[-1] or "asset.bin"
    destination = DOWNLOAD_DIR / filename
    with session.get(url, stream=True, timeout=300) as response:
        response.raise_for_status()
        with open(destination, "wb") as f:
            for chunk in response.iter_content(chunk_size=chunk_size):
                if chunk:
                    f.write(chunk)
    return destination

local_asset = download_asset(asset_url)
local_asset

In [ ]:
def open_dataset(path: Path) -> xr.Dataset:
    if path.suffix in (".nc", ".cdf"):
        return xr.open_dataset(path)
    if path.suffix == ".zarr":
        return xr.open_zarr(path)
    raise ValueError(f"Unsupported asset format: {path.suffix}")

ds = open_dataset(local_asset)
ds

In [ ]:
variable_name = list(ds.data_vars)[0]
da = ds[variable_name]
slice_selectors = {dim: 0 for dim in da.dims if dim in ("time", "depth", "lev")}
subset = da.isel(**slice_selectors)
subset.plot(figsize=(7, 4))
plt.title(f"Quicklook for {variable_name}")
plt.show()